# Sentiment Classification

### Task
* 모델을 바꿔가며 accuracy를 높혀 보자
* 밑에 제시된 여러가지 시도를 해보자
* This code is borrowed from [Hvass-Labs/TensorFlow-Tutorials/20_Natural_Language_Processing](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/20_Natural_Language_Processing.ipynb) which is made of `tf.keras` APIs.
* You can see the detail description or full tutorials: [tutorials link](https://github.com/Hvass-Labs/TensorFlow-Tutorials)

### Dataset
* [IMDB datasets](https://www.imdb.com/interfaces/)
* IMDB 영화사이트에서 50000개의 영화평을 가지고 positive/negative인지 구분

### Base code
* Dataset: train, test로 split (validation data는 `model.fit`할 때 임의로 배정
* Input data shape: (`batch_size`, `max_sequence_length`)
* Output data shape: (`batch_size`, 1)
* Architecture: 
  * `Embedding` - `SimpleRNN` - `Dense (with Sigmoid)`
  * [`tf.keras.layers`](https://www.tensorflow.org/api_docs/python/tf/keras/layers) 사용
* Training
  * `model.fit` 사용
* Evaluation
  * `model.evaluate` 사용 for test dataset

### Try some techniques
* Change model architectures (Custom model)
  * Use another cells (LSTM, GRU, etc.)
  * Use dropout layers
  * Change the `num_words`
  * Change a `padding` method with 'post'
* Data augmentation (if possible)

## Introduction

This tutorial is about a basic form of Natural Language Processing (NLP) called Sentiment Analysis, in which we will try and classify a movie review as either positive or negative.

Consider a simple example: "This movie is not very good." This text ends with the words "very good" which indicates a very positive sentiment, but it is negated because it is preceded by the word "not", so the text should be classified as having a negative sentiment. How can we teach a Neural Network to do this classification?

Another problem is that neural networks cannot work directly on text-data, so we need to convert text into numbers that are compatible with a neural network.

Yet another problem is that a text may be arbitrarily long. The neural networks we have worked with in previous tutorials use fixed data-shapes - except for the first dimension of the data which varies with the batch-size. Now we need a type of neural network that can work on both short and long sequences of text.

## Flowchart

To solve this problem we need several processing steps. First we need to convert the raw text-words into so-called tokens which are integer values. These tokens are really just indices into a list of the entire vocabulary. Then we convert these integer-tokens into so-called embeddings which are real-valued vectors, whose mapping will be trained along with the neural network, so as to map words with similar meanings to similar embedding-vectors. Then we input these embedding-vectors to a Recurrent Neural Network which can take sequences of arbitrary length as input and output a kind of summary of what it has seen in the input. This output is then squashed using a Sigmoid-function to give us a value between 0.0 and 1.0, where 0.0 is taken to mean a negative sentiment and 1.0 means a positive sentiment. This whole process allows us to classify input-text as either having a negative or positive sentiment.

The flowchart of the algorithm is roughly:

<img src="https://user-images.githubusercontent.com/11681225/46912373-d2a3a800-cfae-11e8-8201-ef17b65834f5.png" alt="natural_language_flowchart" style="width: 300px;"/>

## Recurrent Neural Network

The basic building block in a Recurrent Neural Network (RNN) is a Recurrent Unit (RU). There are many different variants of recurrent units such as the rather clunky LSTM (Long-Short-Term-Memory) and the somewhat simpler GRU (Gated Recurrent Unit) which we will use in this tutorial. Experiments in the literature suggest that the LSTM and GRU have roughly similar performance. Even simpler variants also exist and the literature suggests that they may perform even better than both LSTM and GRU, but they are not implemented in Keras which we will use in this tutorial.

The following figure shows the abstract idea of a recurrent unit, which has an internal state that is being updated every time the unit receives a new input. This internal state serves as a kind of memory. However, it is not a traditional kind of computer memory which stores bits that are either on or off. Instead the recurrent unit stores floating-point values in its memory-state, which are read and written using matrix-operations so the operations are all differentiable. This means the memory-state can store arbitrary floating-point values (although typically limited between -1.0 and 1.0) and the network can be trained like a normal neural network using Gradient Descent.

The new state-value depends on both the old state-value and the current input. For example, if the state-value has memorized that we have recently seen the word "not" and the current input is "good" then we need to store a new state-value that memorizes "not good" which indicates a negative sentiment.

The part of the recurrent unit that is responsible for mapping old state-values and inputs to the new state-value is called a gate, but it is really just a type of matrix-operation. There is another gate for calculating the output-values of the recurrent unit. The implementation of these gates vary for different types of recurrent units. This figure merely shows the abstract idea of a recurrent unit. The LSTM has more gates than the GRU but some of them are apparently redundant so they can be omitted.

In order to train the recurrent unit, we must gradually change the weight-matrices of the gates so the recurrent unit gives the desired output for an input sequence. This is done automatically in TensorFlow.

![recurrent_unit](https://user-images.githubusercontent.com/11681225/46912379-f5ce5780-cfae-11e8-85f8-b49cd1283896.png)

### Unrolled Network

Another way to visualize and understand a Recurrent Neural Network is to "unroll" the recursion. In this figure there is only a single recurrent unit denoted RU, which will receive a text-word from the input sequence in a series of time-steps.

The initial memory-state of the RU is reset to zero internally by Keras / TensorFlow every time a new sequence begins.

In the first time-step the word "this" is input to the RU which uses its internal state (initialized to zero) and its gate to calculate the new state. The RU also uses its other gate to calculate the output but it is ignored here because it is only needed at the end of the sequence to output a kind of summary.

In the second time-step the word "is" is input to the RU which now uses the internal state that was just updated from seeing the previous word "this".

There is not much meaning in the words "this is" so the RU probably doesn't save anything important in its internal state from seeing these words. But when it sees the third word "not" the RU has learned that it may be important for determining the overall sentiment of the input-text, so it needs to be stored in the memory-state of the RU, which can be used later when the RU sees the word "good" in time-step 6.

Finally when the entire sequence has been processed, the RU outputs a vector of values that summarizes what it has seen in the input sequence. We then use a fully-connected layer with a Sigmoid activation to get a single value between 0.0 and 1.0 which we interpret as the sentiment either being negative (values close to 0.0) or positive (values close to 1.0).

Note that for the sake of clarity, this figure doesn't show the mapping from text-words to integer-tokens and embedding-vectors, as well as the fully-connected Sigmoid layer on the output.

![unrolled_flowchart](https://user-images.githubusercontent.com/11681225/46912380-0ed70880-cfaf-11e8-8d40-594094f5a905.png)

### 3-Layer Unrolled Network

In this tutorial we will use a Recurrent Neural Network with 3 recurrent units (or layers) denoted RU1, RU2 and RU3 in the "unrolled" figure below.

The first layer is much like the unrolled figure above for a single-layer RNN. First the recurrent unit RU1 has its internal state initialized to zero by Keras / TensorFlow. Then the word "this" is input to RU1 and it updates its internal state. Then it processes the next word "is", and so forth. But instead of outputting a single summary value at the end of the sequence, we use the output of RU1 for every time-step. This creates a new sequence that can then be used as input for the next recurrent unit RU2. The same process is repeated for the second layer and this creates a new output sequence which is then input to the third layer's recurrent unit RU3, whose final output is passed to a fully-connected Sigmoid layer that outputs a value between 0.0 (negative sentiment) and 1.0 (positive sentiment).

Note that for the sake of clarity, the mapping of text-words to integer-tokens and embedding-vectors has been omitted from this figure.


![unrolled_3-layers_flowchart](https://user-images.githubusercontent.com/11681225/46912383-20b8ab80-cfaf-11e8-8da6-db2fa213c43f.png)

### Exploding & Vanishing Gradients

In order to train the weights for the gates inside the recurrent unit, we need to minimize some loss-function which measures the difference between the actual output of the network relative to the desired output.

From the "unrolled" figures above we see that the reccurent units are applied recursively for each word in the input sequence. This means the recurrent gate is applied once for each time-step. The gradient-signals have to flow back from the loss-function all the way to the first time the recurrent gate is used. If the gradient of the recurrent gate is multiplicative, then we essentially have an exponential function.

In this tutorial we will use texts that have more than 500 words. This means the RU's gate for updating its internal memory-state is applied recursively more than 500 times. If a gradient of just 1.01 is multiplied with itself 500 times then it gives a value of about 145. If a gradient of just 0.99 is multiplied with itself 500 times then it gives a value of about 0.007. These are called exploding and vanishing gradients. The only gradients that can survive recurrent multiplication are 0 and 1.

To avoid these so-called exploding and vanishing gradients, care must be made when designing the recurrent unit and its gates. That is why the actual implementation of the GRU is more complicated, because it tries to send the gradient back through the gates without this distortion.

## Imports modules

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import tensorflow as tf
tf.enable_eager_execution()

layers = tf.keras.layers

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Load Data

We will use a data-set consisting of 50000 reviews of movies from IMDB. Keras has a built-in function for downloading a similar data-set (but apparently half the size). However, Keras' version has already converted the text in the data-set to integer-tokens, which is a crucial part of working with natural languages that will also be demonstrated in this tutorial, so we download the actual text-data.

NOTE: The data-set is 84 MB and will be downloaded automatically.

In [ ]:
# if you want to download directly then execute below comment after uncomment
# else use imdb.py and download.py
# !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar xvzf aclImdb_v1.tar.gz
# !mkdir ../../datasets/IMDB
# !mv aclImdb ../../datasets/IMDB
# !rm aclImdb_v1.tar.gz

In [ ]:
import imdb

Change this if you want the files saved in another directory.

In [ ]:
imdb.data_dir = "../../datasets/IMDB/"

Automatically download and extract the files.

In [ ]:
imdb.maybe_download_and_extract()

Load the training- and test-sets.

In [ ]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

In [ ]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

The true "class" is a sentiment of the movie-review. It is a value of 0.0 for a negative sentiment and 1.0 for a positive sentiment.

In [ ]:
# positive sample
index = 1
print("text: {}\nlabel: {}".format(x_train_text[index], y_train[index]))

In [ ]:
# negative sample
index = 20000
print("text: {}\n".format(x_train_text[index]))
print("label: {}".format(y_train[index]))

Combine into one data-set for some uses below.

In [ ]:
data_text = x_train_text + x_test_text

## Tokenizer

A neural network cannot work directly on text-strings so we must convert it somehow. There are two steps in this conversion, the first step is called the "tokenizer" which converts words to integers and is done on the data-set before it is input to the neural network. The second step is an integrated part of the neural network itself and is called the "embedding"-layer, which is described further below.

We may instruct the tokenizer to only use e.g. the 10000 most popular words from the data-set.

In [ ]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

The tokenizer can then be "fitted" to the data-set. This scans through all the text and strips it from unwanted characters such as punctuation, and also converts it to lower-case characters. The tokenizer then builds a vocabulary of all unique words along with various data-structures for accessing the data.

Note that we fit the tokenizer on the entire data-set so it gathers words from both the training- and test-data. This is OK as we are merely building a vocabulary and want it to be as complete as possible. The actual neural network will of course only be trained on the training-set.

In [ ]:
%%time
tokenizer.fit_on_texts(data_text)

If you want to use the entire vocabulary then set `num_words=None` above, and then it will automatically be set to the vocabulary-size here. (This is because of Keras' somewhat awkward implementation.)

In [ ]:
if num_words is None:
  num_words = len(tokenizer.word_index)

In [ ]:
num_words

We can then inspect the vocabulary that has been gathered by the tokenizer. This is ordered by the number of occurrences of the words in the data-set. These integer-numbers are called word indices or "tokens" because they uniquely identify each word in the vocabulary.

In [ ]:
tokenizer.word_index

We can then use the tokenizer to convert all texts in the training-set to lists of these tokens.

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [ ]:
index = 10
print("text: {}\n".format(x_train_text[index]))
print("token: {}".format(x_train_tokens[index]))

## Padding and Truncating Data

The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways of achieving this: (A) Either we ensure that all sequences in the entire data-set have the same length, or (B) we write a custom data-generator that ensures the sequences have the same length within each batch.

Solution (A) is simpler but if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.

So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [ ]:
num_seq_length = np.array([len(tokens) for tokens in x_train_tokens + x_test_tokens])
x_train_seq_length = np.array([len(tokens) for tokens in x_train_tokens], dtype=np.int32)
x_test_seq_length = np.array([len(tokens) for tokens in x_test_tokens], dtype=np.int32)

In [ ]:
num_seq_length.shape

The average number of tokens in a sequence is:

In [ ]:
np.mean(num_seq_length)

The maximum number of tokens in a sequence is:

In [ ]:
np.max(num_seq_length)

The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [ ]:
max_seq_length = np.mean(num_seq_length) + 2 * np.std(num_seq_length)
max_seq_length = int(max_seq_length)
max_seq_length

This covers about 95% of the data-set.

In [ ]:
np.sum(num_seq_length < max_seq_length) / len(num_seq_length)

When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.

So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

In [ ]:
pad = 'pre'
#pad = 'post'

In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_seq_length,
                            padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_seq_length,
                           padding=pad, truncating=pad)

We have now transformed the training-set into one big matrix of integers (tokens) with this shape.
The matrix for the test-set has the same shape.

In [ ]:
print(x_train_pad.shape)
print(x_test_pad.shape)

For example, Let's see the data in each format which are text, tokens, pad

In [ ]:
index = 1
print("text: {}\n".format(x_train_text[index]))
print("token: {}\n".format(x_train_tokens[index]))
print("pad: {}".format(x_train_pad[index]))

## Tokenizer Inverse Map

For some strange reason, the Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. So we make that mapping here.

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
inverse_map

Helper-function for converting a list of tokens back to a string of words.

In [ ]:
def tokens_to_string(tokens):
  # Map from tokens back to words.
  words = [inverse_map[token] for token in tokens if token != 0]

  # Concatenate all words.
  text = " ".join(words)

  return text

For example, Let's see the original text from the data-set.
We can recreate this text except for punctuation and other symbols, by converting the list of tokens back to words.

In [ ]:
index = 0
print("original text:\n{}\n".format(x_train_text[index]))
print("tokens to string:\n{}".format(tokens_to_string(x_train_tokens[index])))

## Setup datasets with `tf.data`

In [ ]:
# Set the hyperparameter set
batch_size = 64
max_epochs = 3
embedding_size = 8

In [ ]:
## create data pipeline with tf.data
#train_dataset = tf.data.Dataset.from_tensor_slices((x_train_pad, x_train_seq_length, y_train))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_pad, y_train))
train_dataset = train_dataset.shuffle(buffer_size = len(x_train_pad) * 3)
train_dataset = train_dataset.repeat(max_epochs)
train_dataset = train_dataset.batch(batch_size = batch_size)
print(train_dataset)

In [ ]:
## create data pipeline with tf.data
#test_dataset = tf.data.Dataset.from_tensor_slices((x_test_pad, x_test_seq_length, y_test))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_pad, y_test))
test_dataset = test_dataset.batch(batch_size = len(x_test_pad))
print(test_dataset)

## Create the Recurrent Neural Network

We are now ready to create the Recurrent Neural Network (RNN).

In [ ]:
model = tf.keras.models.Sequential()

The first layer in the RNN is a so-called Embedding-layer which converts each integer-token into a vector of values. This is necessary because the integer-tokens may take on values between 0 and 10000 for a vocabulary of 10000 words. The RNN cannot work on values in such a wide range. The embedding-layer is trained as a part of the RNN and will learn to map words with similar semantic meanings to similar embedding-vectors, as will be shown further below.

First we define the size of the embedding-vector for each integer-token. In this case we have set it to 8, so that each integer-token will be converted to a vector of length 8. The values of the embedding-vector will generally fall roughly between -1.0 and 1.0, although they may exceed these values somewhat.

The size of the embedding-vector is typically selected between 100-300, but it seems to work reasonably well with small values for Sentiment Analysis.

### Embedding layer
The embedding-layer also needs to know the number of words in the vocabulary (num_words) and the length of the padded token-sequences (max_tokens). We also give this layer a name because we need to retrieve its weights further below.

In [ ]:
model.add(layers.Embedding(input_dim=num_words,
                           output_dim=embedding_size,
                           input_length=max_seq_length,
                           name='layer_embedding'))

We can now add the first Gated Recurrent Unit (GRU) to the network. This will have 16 outputs. Because we will add a second GRU after this one, we need to return sequences of data because the next GRU expects sequences as its input.

In [ ]:
#model.add(layers.GRU(units=16, return_sequences=True))
#model.add(layers.GRU(units=16))
model.add(layers.SimpleRNN(units=16))
#model.add(layers.CuDNNGRU(units=16))

This adds the second GRU with 8 output units. This will be followed by another GRU so it must also return sequences.

In [ ]:
#model.add(layers.GRU(units=8, return_sequences=True))

This adds the third and final GRU with 4 output units. This will be followed by a dense-layer, so it should only give the final output of the GRU and not a whole sequence of outputs.

In [ ]:
#model.add(layers.GRU(units=4))

Add a fully-connected / dense layer which computes a value between 0.0 and 1.0 that will be used as the classification output.

In [ ]:
model.add(layers.Dense(1, activation='sigmoid'))

Use the Adam optimizer with the given learning-rate.

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)

Compile the Keras model so it is ready for training.

In [ ]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

## Train the Recurrent Neural Network

We can now train the model. Note that we are using the data-set with the padded sequences. We use 5% of the training-set as a small validation-set, so we have a rough idea whether the model is generalizing well or if it is perhaps over-fitting to the training-set.

In [ ]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)
#num_train = len(x_train_text)
#model.fit(train_dataset, steps_per_epoch=int(np.ceil(num_train / float(batch_size))), epochs=1)

## Performance on Test-Set

Now that the model has been trained we can calculate its classification accuracy on the test-set.

In [ ]:
%%time
result = model.evaluate(x_test_pad, y_test)
#result = model.evaluate(test_dataset)

In [ ]:
print("Accuracy: {0:.2%}".format(result[1]))

## New Data

Let us try and classify new texts that we make up. Some of these are obvious, while others use negation and sarcasm to try and confuse the model into mis-classifying the text.

In [ ]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]
new_labels = np.array([1, 1, 1, 0, 1, 0, 0, 0])

We first convert these texts to arrays of integer-tokens because that is needed by the model.

In [ ]:
new_data_tokens = tokenizer.texts_to_sequences(texts)

In [ ]:
new_data_seq_length = np.array([len(tokens) for tokens in new_data_tokens], dtype=np.int32)

To input texts with different lengths into the model, we also need to pad and truncate them.

In [ ]:
new_data_pad = pad_sequences(new_data_tokens, maxlen=max_seq_length,
                             padding=pad, truncating=pad)
new_data_pad.shape

We can now use the trained model to predict the sentiment for these texts.

In [ ]:
new_data_predicts = model.predict(new_data_pad)
print(new_data_predicts)

In [ ]:
new_data_predicts = np.squeeze(np.round(new_data_predicts))

In [ ]:
count = 0
for pre, label in zip (new_data_predicts, new_labels):
  pre = pre.astype(np.int32)
  label = label.astype(np.int32)
  if pre == label:
    print("predict: {}, label: {}, results: True".format(pre, label))
    count += 1
  else:
    print("predict: {}, label: {}, results: False".format(pre, label))
    
print("\nAccuracy: {}".format(count / float(len(new_data_predicts))))

## Conclusion

This tutorial showed the basic methods for doing Natural Language Processing (NLP) using a Recurrent Neural Network with integer-tokens and an embedding layer. This was used to do sentiment analysis of movie reviews from IMDB. It works reasonably well if the hyper-parameters are chosen properly. But it is important to understand that this is not human-like comprehension of text. The system does not have any real understanding of the text. It is just a clever way of doing pattern-recognition.

## Exercises

These are a few suggestions for exercises that may help improve your skills with TensorFlow. It is important to get hands-on experience with TensorFlow in order to learn how to use it properly.

You may want to backup this Notebook before making any changes.

* Run more training-epochs. Does it improve performance?
* If your model overfits the training-data, try using dropout-layers and dropout inside the GRU.
* Increase or decrease the number of words in the vocabulary. This is done when the `Tokenizer` is initialized. Does it affect performance?
* Increase the size of the embedding-vectors to e.g. 200. Does it affect performance?
* Try varying all the different hyper-parameters for the Recurrent Neural Network.
* Use Bayesian Optimization from Tutorial #19 to find the best choice of hyper-parameters.
* Use 'post' for padding and truncating in `pad_sequences()`. Does it affect the performance?
* Use individual characters instead of tokenized words as the vocabulary. You can then use one-hot encoded vectors for each character instead of using the embedding-layer.
* Use `model.fit_generator()` instead of `model.fit()` and make your own data-generator, which creates a batch of data using a random subset of `x_train_tokens`. The sequences must be padded so they all match the length of the longest sequence.
* Explain to a friend how the program works.